In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import disk, opening, closing
import glob
import time

In [2]:
def rgb2gray(bgr):
    return np.dot(bgr[...,:3], [0.1140, 0.5870, 0.2989])

def region_growing(image, seed_point, visited, mask, threshold, value_threshold, max_iterations=10000):
    # Get image dimensions
    rows, cols = image.shape[:2]
    # Initialize queue for pixels to visit
    queue = []
    queue.append(seed_point)
    iterations = 0

    # Define 4-connectivity neighbors
    neighbors = [(1, 0), (-1, 0), (0, 1), (0, -1)]

    while queue: # and iterations < max_iterations:
        iterations += 1
        # Get current pixel from queue
        current_point = queue.pop(0)
        mask[current_point] = 1
        visited[current_point] = 1

        for neighbor in neighbors:
            # Calculate neighbor coordinates
            x_neighbor, y_neighbor = current_point[0] + neighbor[0], current_point[1] + neighbor[1]

            # Check if neighbor is within image bounds
            if 0 <= x_neighbor < rows and 0 <= y_neighbor < cols:
                # Check if neighbor pixel is not visited
                if visited[x_neighbor, y_neighbor] == 0: #and (rgb2gray(image[x_neighbor, y_neighbor]) < value_threshold):
                    # Calculate gradient descent
                    gradient = abs(int(rgb2gray(image[current_point])) - int(rgb2gray(image[x_neighbor, y_neighbor])))
                    # Check if gradient is less than threshold
                    if gradient <= threshold:
                        queue.append((x_neighbor, y_neighbor))
                        visited[x_neighbor, y_neighbor] = 1

In [3]:
# Read input image
folder_in = 'imgs_samples/'
images = glob.glob(folder_in + '*.jpg')
imgs = []
for image in images:
    img = cv2.resize(cv2.imread(image), (0, 0), fx=0.5, fy=0.5)
    imgs.append(img)

In [4]:
# Set threshold for region growing
gradient_threshold = 2
value_threshold = 80
masks = []
n = 0
times = []

for input_image in imgs:
    time_start = time.time()
    rows, cols = input_image.shape[:2]
    # Initialize visited matrix, taking only the first 2 dimensions of the input image
    visited = np.zeros((input_image.shape[0], input_image.shape[1]))
    mask = np.zeros((input_image.shape[0], input_image.shape[1]))
    # Initialize segmented image
    segmented_image = input_image.copy()
    # Perform region growing
    for x in range(rows):
        for y in range(cols):
            if (visited[x, y] == 0) and (rgb2gray(input_image[x, y]) < value_threshold):
                region_growing(input_image, (x, y), visited, mask, gradient_threshold, value_threshold)
    mask = closing(opening(mask, disk(7)), disk(7))
    masks.append(mask)
    cv2.imwrite('imgs_samples_mask/mask'+str(n)+'.jpg', mask * 255)
    times.append(time.time() - time_start)
    n += 1
print('Average time: ', sum(times) / len(times), 's')

Average time:  22.039098167419432 s


In [5]:
# load the masks
masks = []
no_of_file = 10
for n in range(no_of_file):
    mask = cv2.imread('imgs_samples_mask/mask'+str(n)+'.jpg', cv2.IMREAD_GRAYSCALE)
    masks.append(mask)

In [6]:
# now we can find the contours
n = 0

for mask, input_image in zip(masks, imgs):
    img_contour = (mask).astype(np.uint8) #convert to uint8 because findContours only accepts uint8
    contours, _ = cv2.findContours(img_contour, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #find contours
    contour = max(contours, key=cv2.contourArea) #get the biggest contour
    img_contour = cv2.cvtColor(img_contour, cv2.COLOR_GRAY2BGR) #convert to BGR because drawContours only accepts BGR
    cv2.drawContours(input_image, contour, -1, (255, 0, 0), 15) #draw the biggest contour

    # find the centroid of the contour
    M = cv2.moments(contour)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    # plot it on the image
    cv2.circle(input_image, (cX, cY), 10, (255, 0, 0), -1)
    cv2.putText(input_image, "centroid", (cX - 30, cY - 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.imwrite('imgs_samples_contour/contour' + str(n) + '.jpg', cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
    n += 1